In [ ]:
# from supervision.assets import download_assets, VideoAssets

# download_assets(VideoAssets.VEHICLES)

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import speed_estimation
import cv2

In [ ]:
model = YOLO("yolov8m.yaml")
names = model.model.names

In [ ]:
cap = cv2.VideoCapture("vehicles.mp4")
assert cap.isOpened()
w,h,fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("speed_estimation.avi",cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (w,h))
line_pts = [(0,h//2),(w,h//2)]
speed_obj = speed_estimation.SpeedEstimator()
speed_obj.set_args(reg_pts=line_pts,
                   names=names,
                   view_img=True)


In [ ]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("video frame is empty or not being read properly")
        break
    tracks = model.track(im0, persist=True, show = False)
    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)
cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [1]:
from ultralytics import YOLO
from ultralytics.solutions import speed_estimation
import cv2
import torch

# torch.cuda.is_available = lambda : False
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0)
model = YOLO("yolov8n.pt")
names = model.model.names

cap = cv2.VideoCapture("vehicles.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, 
cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("speed_estimation.avi",
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (w, h))

line_pts = [(0, h//2), (w,h//2)]

# Init speed-estimation obj
speed_obj = speed_estimation.SpeedEstimator()
speed_obj.set_args(reg_pts=line_pts,
                   names=names,
                   view_img=True)

while cap.isOpened():

    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been \
        successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 1 truck, 99.6ms
Speed: 5.5ms preprocess, 99.6ms inference, 147.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 15.6ms
Speed: 8.0ms preprocess, 15.6ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 16.2ms
Speed: 5.0ms preprocess, 16.2ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 13.8ms
Speed: 4.4ms preprocess, 13.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 21.0ms
Speed: 5.0ms preprocess, 21.0ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 23.0ms
Speed: 4.5ms preprocess, 23.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 17.0ms
Speed: 4.5ms preprocess, 17.0ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 10.5ms
Speed: 3.4ms preprocess, 10.5ms i

KeyboardInterrupt: 

: 

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import heatmap
import cv2
import torch

device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0)
    
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("vehicles.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, 
cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("heatmap_output.avi",
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (320,640))

# Init heatmap
heatmap_obj = heatmap.Heatmap()
heatmap_obj.set_args(colormap=cv2.COLORMAP_PARULA,
                     imw=w,
                     imh=h,
                     view_img=True,
                     shape="circle")

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has \
                been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = heatmap_obj.generate_heatmap(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()